In [ ]:
import os
import time

from TwitterEventDetector import *

In [ ]:
print('Started at:',time.ctime())

## Parameters

In [ ]:
original_tweet_dir = '../data/original_tweets/' # end with '/'
clean_tweet_dir = '../data/cleaned_tweets/without_retweets/' # end with '/'
subwindow_dir = '../data/cleaned_tweets/without_retweets/2012-10-12/' # each file is a subwindow in this folder
event_output_dir = '../Results/'
wiki_titles_file='../data/enwiki-titles-unstemmed.txt'
seg_prob_file='../data/seg_prob_2012_Oct_11-22.json'
common_words_file = '../data/common_words.txt'

# prepocessing
remove_stopwords=True
remove_retweets=True

# segmentation
max_segment_length=4
hashtag_wt=3
entities_only = False

# bursty segment extraction
default_seg_prob=0.0000001
use_retweet_count=True
use_followers_count=True

# clustering
n_neighbors=4
min_cluster_segments=6

## Initialization

In [ ]:
common_words = set() # words to be removed while segmenting

f = open(common_words_file, 'r')
while True:
    line = f.readline()
    if line == '': break
    common_words.add(line.replace('\n',''))
f.close()

In [ ]:
ted = TwitterEventDetector(wiki_titles_file, seg_prob_file, remove_stopwords,remove_retweets, max_segment_length,
                           hashtag_wt, use_retweet_count, use_followers_count, default_seg_prob, common_words, entities_only)

## Tweet Cleaning

In [ ]:
#ted.clean_tweets_in_directory(original_tweet_dir, clean_tweet_dir)

## Segment tweets and create TimeWindow

In [ ]:
print('Reading SubWindows')

subwindow_files = [f.name for f in os.scandir(subwindow_dir) if f.is_file()]

subwindows = []
for subwindow_name in subwindow_files[1:4]: # read given subwindows
    print('SubWindow:',subwindow_name)
    sw = ted.read_subwindow(subwindow_dir + subwindow_name)
    subwindows.append(sw)

print('Done')    

In [ ]:
tw = TimeWindow(subwindows)
print(tw)

In [ ]:
#next_subwindow = ted.read_subwindow(subwindow_dir + subwindow_files[4])
#tw.advance_window(next_subwindow)
#print(tw)

## Bursty Segment Extraction

In [ ]:
bursty_segment_weights = ted.bse.get_bursty_segments(tw)

## Clustering Bursty Segments

In [ ]:
seg_sim = get_seg_similarity(bursty_segment_weights, tw)

In [ ]:
events = get_events(bursty_segment_weights, seg_sim, n_neighbors, min_cluster_segments)

## Save Events to file

In [ ]:
# event_output_dir += time.ctime().replace(':','-') + '/' # unique output directory each time
# os.mkdir(event_output_dir)

# event_no = 0
# for e,avg_score in events:
#     event_no += 1
#     print('\nEVENT:', event_no, ' Avg Bursty Score:', avg_score) 
#     f = open(event_output_dir + str(event_no) + '.txt', 'w')
#     f.write(str(avg_score)+' '+str(e)+'\n')
#     for seg_name in e:
#         print(seg_name) 
#         f.write('SEGMENT:' + seg_name+'\n')
#         for text in set(tw.get_tweets_containing_segment(seg_name)):
#             f.write(text+'\n')
#         f.write('\n')
#     f.close()

In [ ]:
print('Ended at:',time.ctime())

In [ ]:
for n_neighbors,min_cluster_segments in [(2,5), (3,8), (4,10)]:
    print('No. of Neighbors', n_neighbors)
    print('Min Cluster Segments', min_cluster_segments)
    events = get_events(bursty_segment_weights, seg_sim, n_neighbors, min_cluster_segments)
    event_no = 0
    for e,avg_score in events:
        event_no += 1
        print(event_no, e)
    print('-----------------------------------------------------------------------------------------------')